This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.2_tutorials/wallaroo-model-deploy-and-serve/wallaroo-tag-management).

## Wallaroo SDK Tag Tutorial

The following tutorial demonstrates how to use Wallaroo Tags.  Tags are applied to either model versions or pipelines.  This allows organizations to track different versions of models, and search for what pipelines have been used for specific purposes such as testing versus production use.

The following will be demonstrated:

* List all tags in a Wallaroo instance.
* List all tags applied to a model.
* List all tags applied to a pipeline.
* Apply a tag to a model.
* Remove a tag from a model.
* Apply a tag to a pipeline.
* Remove a tag from a pipeline.
* Search for a model version by a tag.
* Search for a pipeline by a tag.

This demonstration provides the following through the Wallaroo Tutorials Github Repository:

* `models/ccfraud.onnx`: a sample model used as part of the [Wallaroo 101 Tutorials](https://docs.wallaroo.ai/wallaroo-101/).

## Prerequisites

* A deployed Wallaroo instance
* The following Python libraries installed:
  * `os`
  * `string`
  * `random`
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.

## Steps

The following steps are performed use to connect to a Wallaroo instance and demonstrate how to use tags with models and pipelines.

### Load Libraries

The first step is to load the libraries used to connect and use a Wallaroo instance.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
import pandas as pd

# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
# Client connection from local Wallaroo instance

wl = wallaroo.Client()

### Set Variables

The following variables are used to create or connect to existing workspace and pipeline.  The model name and model file are set as well.  Adjust as required for your organization's needs.

The methods `get_workspace` and `get_pipeline` are used to either create a new workspace and pipeline based on the variables below, or connect to an existing workspace and pipeline with the same name.  Once complete, the workspace will be set as the current workspace where pipelines and models are used.

In [3]:
workspace_name = f'tagtestworkspace'
pipeline_name = f'tagtestpipeline'
model_name = f'tagtestmodel'
model_file_name = './models/ccfraud.onnx'

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)

{'name': 'tagtestworkspace', 'id': 26, 'archived': False, 'created_by': '65124b18-8382-49af-b3c8-ada3b9df3330', 'created_at': '2024-04-16T20:52:22.403537+00:00', 'models': [], 'pipelines': []}

### Upload Model and Create Pipeline

The `tagtest_model` and `tagtest_pipeline` will be created (or connected if already existing) based on the variables set earlier.

In [5]:
tagtest_model = (wl.upload_model(model_name, 
                                 model_file_name, 
                                 framework=wallaroo.framework.Framework.ONNX)
                                 .configure(tensor_fields=["tensor"])
                )
tagtest_model

Name,tagtestmodel
Version,e3d7cec4-f92c-4c90-b4a7-14252a58860d
File Name,ccfraud.onnx
SHA,bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-16-Apr 20:52:24


In [6]:
tagtest_pipeline = wl.build_pipeline(pipeline_name)
tagtest_pipeline

name,tagtestpipeline
created,2024-04-16 20:52:25.682271+00:00
last_updated,2024-04-16 20:52:25.682271+00:00
deployed,(none)
arch,None
accel,None
tags,
versions,2e303938-dc76-4dbd-a88a-d515cf2430b1
steps,
published,False


### List Pipeline and Model Tags

This tutorial assumes that no tags are currently existing, but that can be verified through the Wallaroo client `list_pipelines` and `list_models` commands.  For this demonstration, it is recommended to use unique tags to verify each example.

In [7]:
wl.list_pipelines()

AttributeError: 'NoneType' object has no attribute 'get'

[{'name': 'tagtestpipeline', 'create_time': datetime.datetime(2024, 4, 16, 20, 52, 25, 682271, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'retail-inv-tracker-edge-obs', 'create_time': datetime.datetime(2024, 4, 16, 20, 52, 25, 348600, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'apiinferenceexamplepipeline', 'create_time': datetime.datetime(2024, 4, 16, 20, 35, 54, 584977, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'sdkinferenceexamplepipeline', 'create_time': datetime.datetime(2024, 4, 16, 20, 27, 30, 246869, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'alohapipeline', 'create_time': datetime.datetime(2024, 4, 16, 20, 15, 1, 333941, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'multi-io-example', 'create_time': datetime.datetime(2024, 4, 16, 20, 13, 37, 949936, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'mlflowstatsmodelpipeline', 'create_time': datetime.datetime(2024, 4, 16, 20, 7, 55, 551305, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'imdbpipeline', '

In [8]:
wl.list_models()

Name,# of Versions,Owner ID,Last Updated,Created At
tagtestmodel,1,"""""",2024-04-16 20:52:24.844870+00:00,2024-04-16 20:52:24.844870+00:00


### Create Tag

Tags are created with the Wallaroo client command `create_tag(String tagname)`.  This creates the tag and makes it available for use.

The tag will be saved to the variable `currentTag` to be used in the rest of these examples.

In [9]:
# Now we create our tag
currentTag = wl.create_tag("My Great Tag")

### List Tags

Tags are listed with the Wallaroo client command `list_tags()`, which shows all tags and what models and pipelines they have been assigned to.  Note that if a **tag has not been assigned**, it will not be displayed.

In [10]:
# List all tags

wl.list_tags()

id,tag,models,pipelines
1,My Great Tag,[],[]


### Assign Tag to a Model

Tags are assigned to a model through the Wallaroo Tag `add_to_model(model_id)` command, where `model_id` is the model's numerical ID number.  The tag is applied to the most current version of the model.

For this example, the `currentTag` will be applied to the `tagtest_model`.  All tags will then be listed to show it has been assigned to this model.

In [11]:
# add tag to model

currentTag.add_to_model(tagtest_model.id())

{'model_id': 33, 'tag_id': 1}

In [12]:
# list all tags to verify

wl.list_tags()

id,tag,models,pipelines
1,My Great Tag,"[('tagtestmodel', ['e3d7cec4-f92c-4c90-b4a7-14252a58860d'])]",[]


### Search Models by Tag

Model versions can be searched via tags using the Wallaroo Client method `search_models(search_term)`, where `search_term` is a string value.  All models versions containing the tag will be displayed.  In this example, we will be using the text from our tag to list all models that have the text from `currentTag` in them.

In [13]:
# Search models by tag

wl.search_models('My Great Tag')

name,version,file_name,image_path,arch,accel,last_update_time
tagtestmodel,e3d7cec4-f92c-4c90-b4a7-14252a58860d,ccfraud.onnx,None,x86,none,2024-04-16 20:52:24.844870+00:00


### Remove Tag from Model

Tags are removed from models using the Wallaroo Tag `remove_from_model(model_id)` command.

In this example, the `currentTag` will be removed from `tagtest_model`.  A list of all tags will be shown with the `list_tags` command, followed by searching the models for the tag to verify it has been removed.

In [14]:
### remove tag from model

currentTag.remove_from_model(tagtest_model.id())

{'model_id': 33, 'tag_id': 1}

In [15]:
# list all tags to verify it has been removed from `tagtest_model`.

wl.list_tags()

id,tag,models,pipelines
1,My Great Tag,[],[]


In [16]:
# search models for currentTag to verify it has been removed from `tagtest_model`.

wl.search_models('My Great Tag')

[]

### Add Tag to Pipeline

Tags are added to a pipeline through the Wallaroo Tag `add_to_pipeline(pipeline_id)` method, where `pipeline_id` is the pipeline's integer id.

For this example, we will add `currentTag` to `testtest_pipeline`, then verify it has been added through the `list_tags` command and `list_pipelines` command.

In [17]:
# add this tag to the pipeline
currentTag.add_to_pipeline(tagtest_pipeline.id())

{'pipeline_pk_id': 65, 'tag_pk_id': 1}

In [18]:
# list tags to verify it was added to tagtest_pipeline

wl.list_tags()


id,tag,models,pipelines
1,My Great Tag,[],"[('tagtestpipeline', ['2e303938-dc76-4dbd-a88a-d515cf2430b1'])]"


In [19]:
# get all of the pipelines to show the tag was added to tagtest-pipeline

wl.list_pipelines()

AttributeError: 'NoneType' object has no attribute 'get'

[{'name': 'tagtestpipeline', 'create_time': datetime.datetime(2024, 4, 16, 20, 52, 25, 682271, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'retail-inv-tracker-edge-obs', 'create_time': datetime.datetime(2024, 4, 16, 20, 52, 25, 348600, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'apiinferenceexamplepipeline', 'create_time': datetime.datetime(2024, 4, 16, 20, 35, 54, 584977, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'sdkinferenceexamplepipeline', 'create_time': datetime.datetime(2024, 4, 16, 20, 27, 30, 246869, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'alohapipeline', 'create_time': datetime.datetime(2024, 4, 16, 20, 15, 1, 333941, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'multi-io-example', 'create_time': datetime.datetime(2024, 4, 16, 20, 13, 37, 949936, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'mlflowstatsmodelpipeline', 'create_time': datetime.datetime(2024, 4, 16, 20, 7, 55, 551305, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'imdbpipeline', '

### Search Pipelines by Tag

Pipelines can be searched through the Wallaroo Client `search_pipelines(search_term)` method, where `search_term` is a string value for tags assigned to the pipelines.

In this example, the text "My Great Tag" that corresponds to `currentTag` will be searched for and displayed.

In [20]:
wl.search_pipelines('My Great Tag')

name,version,creation_time,last_updated_time,deployed,tags,steps
tagtestpipeline,2e303938-dc76-4dbd-a88a-d515cf2430b1,2024-16-Apr 20:52:25,2024-16-Apr 20:52:25,(unknown),My Great Tag,


### Remove Tag from Pipeline

Tags are removed from a pipeline with the Wallaroo Tag `remove_from_pipeline(pipeline_id)` command, where `pipeline_id` is the integer value of the pipeline's id.

For this example, `currentTag` will be removed from `tagtest_pipeline`.  This will be verified through the `list_tags` and `search_pipelines` command.

In [21]:
## remove from pipeline
currentTag.remove_from_pipeline(tagtest_pipeline.id())

{'pipeline_pk_id': 65, 'tag_pk_id': 1}

In [22]:
wl.list_tags()

id,tag,models,pipelines
1,My Great Tag,[],[]


In [23]:
## Verify it was removed
wl.search_pipelines('My Great Tag')

[]